In [2]:
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d disham993/9000-movies-dataset

100% 1.70M/1.70M [00:01<00:00, 1.87MB/s]
100% 1.70M/1.70M [00:01<00:00, 1.67MB/s]


In [3]:
!unzip 9000-movies-dataset.zip

Archive:  9000-movies-dataset.zip
  inflating: mymoviedb.csv           


In [28]:
import pandas as pd

dataset = pd.read_csv("mymoviedb.csv",lineterminator="\n")
dataset.head()

,Release_Date,Title,Overview,Popularity,Vote_Count,Vote_Average,Original_Language,Genre,Poster_Url
0,2021-12-15,Spider-Man: No Way Home,Peter Parker is unmasked and no longer able to...,5083.954,8940,8.3,en,"Action, Adventure, Science Fiction",https://image.tmdb.org/t/p/original/1g0dhYtq4i...
1,2022-03-01,The Batman,"In his second year of fighting crime, Batman u...",3827.658,1151,8.1,en,"Crime, Mystery, Thriller",https://image.tmdb.org/t/p/original/74xTEgt7R3...
2,2022-02-25,No Exit,Stranded at a rest stop in the mountains durin...,2618.087,122,6.3,en,Thriller,https://image.tmdb.org/t/p/original/vDHsLnOWKl...
3,2021-11-24,Encanto,"The tale of an extraordinary family, the Madri...",2402.201,5076,7.7,en,"Animation, Comedy, Family, Fantasy",https://image.tmdb.org/t/p/original/4j0PNHkMr5...
4,2021-12-22,The King's Man,As a collection of history's worst tyrants and...,1895.511,1793,7.0,en,"Action, Adventure, Thriller, War",https://image.tmdb.org/t/p/original/aq4Pwv5Xeu...


In [29]:
data = dataset[['Title','Overview','Genre']]
data.head()

,Title,Overview,Genre
0,Spider-Man: No Way Home,Peter Parker is unmasked and no longer able to...,"Action, Adventure, Science Fiction"
1,The Batman,"In his second year of fighting crime, Batman u...","Crime, Mystery, Thriller"
2,No Exit,Stranded at a rest stop in the mountains durin...,Thriller
3,Encanto,"The tale of an extraordinary family, the Madri...","Animation, Comedy, Family, Fantasy"
4,The King's Man,As a collection of history's worst tyrants and...,"Action, Adventure, Thriller, War"


In [30]:
data.shape

(9827, 3)

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
data['Overview'] = data['Overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(data['Overview'])
tfidf_matrix.shape

<ipython-input-31-c40f06c37b7e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Overview'] = data['Overview'].fillna('')


(9827, 28483)

In [34]:
# model learning
from sklearn.metrics.pairwise import linear_kernel

base = linear_kernel(tfidf_matrix, tfidf_matrix)

In [35]:
# mapping movie name to the indices
indices = pd.Series(data.index,index=data['Title']).drop_duplicates()
indices[0:10]

Title
Spider-Man: No Way Home    0
The Batman                 1
No Exit                    2
Encanto                    3
The King's Man             4
The Commando               5
Scream                     6
Kimi                       7
Fistful of Vengeance       8
Eternals                   9
dtype: int64

In [80]:
def get_recommendations(title, cosine_sim=base):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores.sort(key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:4]
    movie_indices = [i[0] for i in sim_scores]
    similar_movies = data['Title'].iloc[movie_indices].tolist()

    return similar_movies


In [81]:
get_recommendations("Encanto")

['Out of the Dark', 'The Secret of the Magic Gourd', 'No Exit']